In [1]:
using SparseArrays
using LinearAlgebra
using NLopt
using PyPlot
using KrylovKit
using FiniteDifferences
using TopologyOptimizationHelper

Precompiling TopologyOptimizationHelper
  ✓ TopologyOptimizationHelper
  1 dependency successfully precompiled in 8 seconds. 39 already precompiled.


In [24]:
# Our favorite parameters for testing
# Run to recompute random variables
L = 20
ε = rand(480) .* 11 .+ 1
δε = randn(480) * 1e-6
ω = 2π
δω = randn() * 1e-6
dir = randn(length(ε))

A, x = Maxwell1d(L, ε, ω)
new_ε_A, _ = Maxwell1d(L, ε + δε, ω)
new_ω_A, _ = Maxwell1d(L, ε, ω + δω)

M = length(x)
b = zeros(M)
b[M÷2] = 1;

In [25]:
# Exact ∇_ε_LDOS versus finite difference
LDOS, ∇LDOS = ∇_ε_LDOS(A, b, ω)
new_LDOS, _ = ∇_ε_LDOS(new_ε_A, b, ω)

@show new_LDOS - LDOS
∇LDOS' * δε

new_LDOS - LDOS = -2.5824369474093918e-15


-2.5824520927242653e-15

In [26]:
# Exact ∇_ε_ω₀ versus finite difference
val, _ = Arnoldi_eig(A, ε, ω, b)
new_val, _ = Arnoldi_eig(new_A, ε + δε, ω, b)
gradient = Eigengradient(A, ε, ω, b)
@show sqrt(new_val) - sqrt(val)
@show dot(δε, gradient)

# Exact ∇_ε_ω₀ versus Richardson extrapolation 
f = α -> Maxwell_omega(L, ε + α * dir, ω, b)
@show extrapolate_fdm(central_fdm(2, 1), f, 0)
dir' * gradient

sqrt(new_val) - sqrt(val) = 0.14056922149473294 - 0.003365397847981965im
dot(δε, gradient) = 1.4014226055812528e-7 + 9.571107022653217e-13im
extrapolate_fdm(central_fdm(2, 1), f, 0) = (-0.015876694860308238 + 2.0723175871000079e-7im, 4.72337530883195e-13)


-0.015876694860008013 + 2.0723175863034393e-7im

In [28]:
# Exact ∇_ω_LDOS versus finite difference
∇LDOS = ∇_ω_LDOS(A, b, ε, ω)
LDOS = Just_LDOS(L, ε, ω, b)
new_LDOS = Just_LDOS(L, ε, ω + δω, b; ω_pml=ω)

@show new_LDOS - LDOS
∇LDOS * real(δω)

new_LDOS - LDOS = -4.757748480451429e-15


1-element Vector{Float64}:
 -4.757801000413143e-15

In [29]:
# Exact improved gradient versus finite difference
ω₀ = sqrt(Arnoldi_eig(A, ε, ω, b)[1])
A₀, _ = Maxwell1d(L, ε, real(ω₀); ω_pml=ω)
LDOS, ∇LDOS = Improved_∇_ε_LDOS(A₀, ε, real(ω₀), b; ω_pml=ω)
new_LDOS =  Just_Improved_LDOS(L, ε + δε, ω, b)
@show new_LDOS - LDOS
@show ∇LDOS' * δε

# Exact improved gradient versus Richardson extrapolation
f = α -> Just_Improved_LDOS(L, ε + α * dir, ω, b)
@show extrapolate_fdm(central_fdm(2, 1), f, 0)
dir' * ∇LDOS

new_LDOS - LDOS = 0.000191539875061153
∇LDOS' * δε = 0.00019153980706303634
extrapolate_fdm(central_fdm(2, 1), f, 0) = (-136.94919726108344, 3.6314787621449796e-8)


-136.94919726302587